In [0]:
#anomaly detection
#credit_card fraud analysis
#Unusual behavior 
#import libraries 
#Courtesy:https://www.kaggle.com/pavansanagapati/anomaly-detection-credit-card-fraud-analysis
import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams


In [7]:
from google.colab import files
uploaded = files.upload()

Saving creditcard.csv to creditcard.csv


In [2]:
!ls

creditcard.csv	sample_data


In [0]:
#read the data
data = pd.read_csv('creditcard.csv',sep=',')

In [4]:
#total data
data.shape

(284807, 31)

In [5]:
#determine number of fraud in the dataset 
#fraud=1, solid data=0
Fraud = data[data['Class']==1]

Normal = data[data['Class']==0]

Fraud.shape

(492, 31)

In [6]:
data1= data.sample(frac = 0.1,random_state=1)

data1.shape

(28481, 31)

In [0]:
# Create a trace

trace = go.Scatter(
    x = Fraud.Time,
    y = Fraud.Amount,
    mode = 'markers'
)
data = [trace]

In [8]:
data1.shape

(28481, 31)

In [0]:
Fraud = data1[data1['Class']==1]
Valid = data1[data1['Class']==0]
outlier_fraction = len(Fraud)/float(len(Valid))

In [18]:
print(outlier_fraction)
print("Fraud Cases : {}".format(len(Fraud)))
print("Valid Cases : {}".format(len(Valid)))

0.0017234102419808666
Fraud Cases : 49
Valid Cases : 28432


In [20]:
#Get the X and target data 
columns = data1.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["Class"]]
# Store the variable we are predicting 
target = "Class"
# Define a random state 
state = np.random.RandomState(42)
X = data1[columns]
Y = data1[target]
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

(28481, 30)
(28481,)


In [0]:
X.head()

In [0]:
#isolation forest 
classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X), 
                                       contamination=outlier_fraction,random_state=state, verbose=0),
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto', 
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=outlier_fraction)
    #"Support Vector Machine":OneClassSVM(kernel='rbf', degree=3, gamma=0.1,nu=0.05, 
                                        # max_iter=-1, random_state=state)
   
}

In [0]:
!pip install -U scikit-learn

In [0]:
!pip install numpy

In [33]:
n=len(Fraud)
clf_name = "Isolation Forest"
clf.fit(X,Y)

LocalOutlierFactor(contamination=0.0017234102419808666)

In [38]:
y_pred=clf.fit_predict(X)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [39]:
 #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
y_pred[y_pred == 1] = 0
y_pred[y_pred == -1] = 1
n_errors = (y_pred != Y).sum()
    # Run Classification Metrics
print("{}: {}".format(clf_name,n_errors))
print("Accuracy Score :")
print(accuracy_score(Y,y_pred))
print("Classification Report :")
print(classification_report(Y,y_pred))

Isolation Forest: 97
Accuracy Score :
0.9965942207085425
Classification Report :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.02      0.02      0.02        49

    accuracy                           1.00     28481
   macro avg       0.51      0.51      0.51     28481
weighted avg       1.00      1.00      1.00     28481

